In [60]:
import pandas as pd
from os import path
from sklearn.preprocessing import OneHotEncoder

In [61]:
data_path = path.abspath("../data/US_Superstore_data.xls")
data = pd.read_excel(data_path)
# Country -> only have United States values
data = data.drop(columns=['Row ID', 'Order ID', 'Customer Name', 'Postal Code', 'Product ID', 'Product Name', 'Customer ID', 'Order Date', 'Ship Date', 'Country', "City"])
data

,Ship Mode,Segment,State,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,Second Class,Consumer,Kentucky,South,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,Second Class,Consumer,Kentucky,South,Furniture,Chairs,731.9400,3,0.00,219.5820
2,Second Class,Corporate,California,West,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,Standard Class,Consumer,Florida,South,Furniture,Tables,957.5775,5,0.45,-383.0310
4,Standard Class,Consumer,Florida,South,Office Supplies,Storage,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...
9989,Second Class,Consumer,Florida,South,Furniture,Furnishings,25.2480,3,0.20,4.1028
9990,Standard Class,Consumer,California,West,Furniture,Furnishings,91.9600,2,0.00,15.6332
9991,Standard Class,Consumer,California,West,Technology,Phones,258.5760,2,0.20,19.3932
9992,Standard Class,Consumer,California,West,Office Supplies,Paper,29.6000,4,0.00,13.3200


In [62]:
"""
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="distancias") # Replace "my_geopy_app" with a descriptive name for your application

# 39.828175 e 98.5795, centro do EUA
def distancia_do_centro(cidade:str):
  location = geolocator.geocode(cidade)
  if location:
        latitude = location.latitude
        longitude = location.longitude
        distancia_la = 39.828175 - latitude
        distancia_lo = 98.5795 - longitude
        distancia = (distancia_la**2 + distancia_lo**2)** 0.5
        
        return(distancia)
  
dict_coords = {}
for c in data["City"].unique():
    print(c)
    dict_coords[c] = distancia_do_centro(c)

data['City'] = data['City'].map(dict_coords)
data['City']=(data['City']-data['City'].mean())/data['City'].std()
"""


'\nfrom geopy.geocoders import Nominatim\n\ngeolocator = Nominatim(user_agent="distancias") # Replace "my_geopy_app" with a descriptive name for your application\n\n# 39.828175 e 98.5795, centro do EUA\ndef distancia_do_centro(cidade:str):\n  location = geolocator.geocode(cidade)\n  if location:\n        latitude = location.latitude\n        longitude = location.longitude\n        distancia_la = 39.828175 - latitude\n        distancia_lo = 98.5795 - longitude\n        distancia = (distancia_la**2 + distancia_lo**2)** 0.5\n\n        return(distancia)\n\ndict_coords = {}\nfor c in data["City"].unique():\n    print(c)\n    dict_coords[c] = distancia_do_centro(c)\n\ndata[\'City\'] = data[\'City\'].map(dict_coords)\ndata[\'City\']=(data[\'City\']-data[\'City\'].mean())/data[\'City\'].std()\n'

Normalização

In [63]:
data['Sales']=(data['Sales']-data['Sales'].mean())/data['Sales'].std()
data['Profit']=(data['Profit']-data['Profit'].mean())/data['Profit'].std()
data

,Ship Mode,Segment,State,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,Second Class,Consumer,Kentucky,South,Furniture,Bookcases,0.051508,2,0.00,0.056590
1,Second Class,Consumer,Kentucky,South,Furniture,Chairs,0.805593,3,0.00,0.815013
2,Second Class,Corporate,California,West,Office Supplies,Labels,-0.345350,2,0.00,-0.092997
3,Standard Class,Consumer,Florida,South,Furniture,Tables,1.167630,5,0.45,-1.757397
4,Standard Class,Consumer,Florida,South,Office Supplies,Storage,-0.332919,2,0.20,-0.111587
...,...,...,...,...,...,...,...,...,...,...
9989,Second Class,Consumer,Florida,South,Furniture,Furnishings,-0.328298,3,0.20,-0.104816
9990,Standard Class,Consumer,California,West,Furniture,Furnishings,-0.221258,2,0.00,-0.055595
9991,Standard Class,Consumer,California,West,Technology,Phones,0.046078,2,0.20,-0.039544
9992,Standard Class,Consumer,California,West,Office Supplies,Paper,-0.321315,4,0.00,-0.065470


 One-Hot Encoding

In [64]:
encoder_states = OneHotEncoder(sparse_output=False)
states = encoder_states.fit_transform(data[['State']])
states_df = pd.DataFrame(states, columns=encoder_states.get_feature_names_out(['State']))

encoder_segment= OneHotEncoder(sparse_output=False)
segment = encoder_segment.fit_transform(data[['Segment']])
segment_df = pd.DataFrame(segment, columns=encoder_segment.get_feature_names_out(['Segment']))

encoder_ship_m= OneHotEncoder(sparse_output=False)
ship_m = encoder_ship_m.fit_transform(data[['Ship Mode']])
ship_df = pd.DataFrame(ship_m, columns=encoder_ship_m.get_feature_names_out(['Ship Mode']))

encoder_region= OneHotEncoder(sparse_output=False)
region = encoder_region.fit_transform(data[['Region']])
region_df = pd.DataFrame(region, columns=encoder_region.get_feature_names_out(['Region']))

encoder_category= OneHotEncoder(sparse_output=False)
category = encoder_category.fit_transform(data[['Category']])
category_df = pd.DataFrame(category, columns=encoder_category.get_feature_names_out(['Category']))

encoded_data = pd.concat([states_df, segment_df, ship_df, region_df, category_df, data["Sales"], data["Quantity"], data["Discount"], data["Profit"]], axis=1)
#encoded_data = pd.concat([segment_df, ship_df, category_df, data["Sales"], data["Quantity"], data["Discount"], data["Profit"]], axis=1)
encoded_data

,State_Alabama,State_Arizona,State_Arkansas,State_California,State_Colorado,State_Connecticut,State_Delaware,State_District of Columbia,State_Florida,State_Georgia,...,Region_East,Region_South,Region_West,Category_Furniture,Category_Office Supplies,Category_Technology,Sales,Quantity,Discount,Profit
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.051508,2,0.00,0.056590
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.805593,3,0.00,0.815013
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,-0.345350,2,0.00,-0.092997
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.167630,5,0.45,-1.757397
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,-0.332919,2,0.20,-0.111587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,-0.328298,3,0.20,-0.104816
9990,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,-0.221258,2,0.00,-0.055595
9991,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.046078,2,0.20,-0.039544
9992,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,-0.321315,4,0.00,-0.065470


In [65]:
data_path = path.abspath("../data/processed_data/US_Superstore_data.xls")
encoded_data.to_csv(str(data_path), index=False)